In [1]:

from src.basic_functions import get_prediction
from transformers import pipeline
from src.utils import save_to_references
from metrics.correctness import correctness
from metrics.consistency import consistency
from metrics.plausibility import plausibility
from metrics.qag import qag_score
from metrics.contextual import contextual_faithfulness
from metrics.counterfactual import counterfactual_faithfulness
from metrics.faithfulness import faithfulness
from metrics.trustworthiness import lext

c:\Users\skrit\miniconda3\envs\medllm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\skrit\miniconda3\envs\medllm\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Target Models and Helper Keys Initialization 

Add the ner-pipe entitiy from huggingface using which you want your explanations will be tagged. 
The below ner_pipe is a medical NER model. Other domain-specific NER models can be appropriately used for evaluating explanations. 

In [ ]:
ner_pipe = pipeline("token-classification", model="Clinical-AI-Apollo/Medical-NER", aggregation_strategy='simple')

This evaluation uses groq API for accessing bigger models like llama3-70b. Create an API key from Groq and replace it in the placeholder below. You can also replace the bigger model by going to [src/basic_functions.py](src/basic_functions.py) and replacing it with any othe model from groq

In [ ]:
groq_api= "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"
model ="llama3.1"

Fill in the question and context, along with the ground label and explanation for computing the LeXT Score

In [ ]:
question = "I have a fever, should I take a paracetamol. Yes/No?"
context= None 
ground_label= "Yes"
ground_explanation= "Paracetamol is the recommended and is effective for reduction of fever in adults and children. However it is always recommended to consult with a healthcare professional before taking any medication. "

# Computing the Overall LExT Score
Run the below function. All the relevant scores will be printed and the results along with the reference data for further analysis will be stored in references.csv. Make sure that the models are pulled locally on ollama and the api keys and pipelines are correct. 

In [ ]:
LExT = lext(context, question, ground_explanation, ground_label, model, groq_api, ner_pipe)

Fever, paracetamal
QAG: 1.0, Counterfactual: 1.0, Contextual: 0, Faithfulness: 0.6666666666666666

LExT (Language Explanation Trustworthiness) Score: 0.0
All model outputs are saved in data/references.csv


# Computing Individual Metrics

To compute any individual or aggregate set of metrics, run the below code to get the sample prediction and create a row reference dictionary that can be used to access and save data by other helper functions. 

In [ ]:
label, explanation = get_prediction(context, question, model, groq_api, include_context="False")

In [ ]:
row_reference = {
        "ground_context": context,
        "ground_question": question,
        "ground_explanation": ground_explanation,
        "ground_label":ground_label,
        "predicted_explanation": explanation,
        "predicted_label" : label
    }